# D7047E - Lab 1
Muhammadjon Hamraliev, Johanna Hultgren, Saga Lindkvist, Alfred Olsson, Johanna Söderström

## Task 1.1

Loading the data.


In [ ]:
#Muhammadjon Hamraliev
#API KEY: b660489761915cbeae2e964a70f9a20b9ec4d250
#Mounting the drive
from google.colab import drive
drive.mount('/content/drive/')

#Importing the necessary packages
import torch
import copy
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')




def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+','', regex=True) # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}','', regex=True) # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','') # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True) # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_.loc[len(df_)] = {
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent)
        }
    return data




# get data, pre-process and split
data = pd.read_csv("/content/drive/MyDrive/Data/amazon_cells_labelled.txt", delimiter='\t', header=None)
#print(type(data))
data.columns = ['Sentence', 'Class']
data['index'] = data.index # add new column index
columns = ['index', 'Class', 'Sentence']
# split the data into training, validation, and test splits
training_data, valtest_data, training_labels, valtest_labels = train_test_split(
    data['Sentence'].values.astype('U'),
    data['Class'].values.astype('int32'),
    test_size=0.40,
    random_state=0,
    shuffle=True
)

test_data, validation_data, test_labels, validation_labels = train_test_split(
    valtest_data,
    valtest_labels,
    test_size=0.50,
    random_state=0,
    shuffle=True
)

#print(f"Training data: {training_data[0]}", f"Training Label: {training_labels[0]}")


#print(training_data[0])
#print(training_labels[0])

#print("training data", train_x_tensor.size())
#print("training labels", train_y_tensor[0])




#print("Vector size: ",vocab_size)


#print("One Data", (training_data[0]))
#print("One Label", (training_labels[0]))
#print("Test Data", test_data[0])


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#BIG data loading in.



# get data, pre-process and split
BIG_data = pd.read_csv("/content/drive/MyDrive/Data/amazon_cells_labelled_LARGE_25K.txt", delimiter='\t', header=None)
#print(type(data))
BIG_data.columns = ['Sentence', 'Class']
BIG_data['index'] = BIG_data.index # add new column index
# split the data into training, validation, and test splits
BIG_training_data, BIG_valtest_data, BIG_training_labels, BIG_valtest_labels = train_test_split(
    BIG_data['Sentence'].values.astype('U'),
    BIG_data['Class'].values.astype('int32'),
    test_size=0.40,
    random_state=0,
    shuffle=True
)

BIG_test_data, BIG_validation_data, BIG_test_labels, BIG_validation_labels = train_test_split(
    BIG_valtest_data,
    BIG_valtest_labels,
    test_size=0.50,
    random_state=0,
    shuffle=True
)

In [ ]:
print(len(BIG_training_data))
#

15000


Creating the model

In [ ]:
#Little Data training

#Muhammadjon Hamraiev
class ANNModel():

    training_losses, validation_losses = [], []
    best_model = None

    def __init__(self, in_features, learning_rate):
        self.model = nn.Sequential(
            nn.Linear(in_features, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 2)
        )
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay = 0.0009)

        self.loss_fn = torch.nn.CrossEntropyLoss()



    def training(self, train_data, train_labels, validation_data, validation_labels, epochs, device):
        num_epochs = epochs
        best_loss = float("inf")

        self.model.train()
        self.model.to(device)
        train_data = train_data.to(device)
        train_labels = train_labels.to(device)
        validation_data = validation_data.to(device)
        validation_labels = validation_labels.to(device)



        for epoch in range(num_epochs):
            training_loss, validation_loss = 0, 0

            # To be adjusted based off how dataloaders are implemented. For now assume each batch holds the images and the labels
            #for i, data in enumerate(train_data):


            prediction = self.model(train_data)
                #print(torch.argmax(prediction.data))
                #prediction = torch.argmax(prediction.data)
            #print("prediction", prediction)
            #print("train_labels", train_labels)
            #print("prediction type: ", prediction.size())
            #print("train_labels type: ", train_labels.size())

            self.optimizer.zero_grad()

            loss = self.loss_fn(prediction, train_labels)

            loss.backward()

            self.optimizer.step()
            #print(f"Training Loss: {loss.item()}")
            self.training_losses.append(loss.item())

            correct = 0
            # Validation loop
            with torch.no_grad():
                prediction = self.model(validation_data)

                loss = self.loss_fn((prediction), (validation_labels))
                validation_loss += loss.item()
                #predicted = torch.argmax(prediction.data)
                #print("Predicted", predicted)
                #print("Validation label", validation_labels[i])
                for i in range(len(validation_labels)):
                    #print(f"Prediction {prediction[0]}")
                    predicted = torch.argmax(prediction[i])
                    #print(f"Predicted {predicted}")
                    if predicted == validation_labels[i]:
                        correct += 1

                self.validation_losses.append(validation_loss)
            accuracy = correct / len(validation_labels)
            if validation_loss < best_loss:
                    best_loss = validation_loss
                    self.best_model = copy.deepcopy(self.model)
                    #print(type(self.best_model))
                    print("Best model accuracy", accuracy)
                    print("Best model found")


            print(f"Epoch: {epoch+1}\nTraining Loss: {self.training_losses[epoch]}\nValidation loss: {self.validation_losses[epoch]}\nBest Loss: {best_loss}\nValidation accuracy: {accuracy}")

    def getHistoryAsString(self):
        text = ""

        for i in range(len(self.training_losses)):
            text += f"Epoch: {i+1}\nTraining Loss: {self.training_losses[i]}\nValidation loss: {self.validation_losses[i]}\n\n"
        return text


TrainingData = pd.DataFrame({'index': pd.RangeIndex(600),'Sentence': training_data, 'Class': training_labels})
ValidationData = pd.DataFrame({'index': pd.RangeIndex(200), 'Sentence': validation_data, 'Class': validation_labels})
TestData = pd.DataFrame({'index': pd.RangeIndex(200), 'Sentence': test_data, 'Class': test_labels})


#Pre-Processing the data
processedTrainingData = preprocess_pandas(TrainingData, columns)
processedValidationData = preprocess_pandas(ValidationData, columns)
processedTestData = preprocess_pandas(TestData, columns)





# vectorize data using TFIDF and transform for PyTorch for scalability
# words used less often across all documents, and used frequently within one document gets a higher score (weights)
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
anntraining_data = word_vectorizer.fit_transform(processedTrainingData['Sentence']) # transform texts to sparse matrix
anntraining_data = anntraining_data.todense() # convert to dense matrix for Pytorch
vocab_size = len(word_vectorizer.vocabulary_)
annvalidation_data = word_vectorizer.transform(processedValidationData['Sentence'])
annvalidation_data = annvalidation_data.todense()
anntest_data = word_vectorizer.transform(processedTestData['Sentence'])
anntest_data = anntest_data.todense()
train_x_tensor = torch.from_numpy(np.array(anntraining_data)).type(torch.FloatTensor)
train_y_tensor = torch.from_numpy(np.array(training_labels)).type(torch.LongTensor)
validation_x_tensor = torch.from_numpy(np.array(annvalidation_data)).type(torch.FloatTensor)
validation_y_tensor = torch.from_numpy(np.array(validation_labels)).type(torch.LongTensor)
test_x_tensor = torch.from_numpy(np.array(anntest_data)).type(torch.FloatTensor)
test_y_tensor = torch.from_numpy(np.array(test_labels)).type(torch.LongTensor)




device = "cuda" if torch.cuda.is_available() else "cpu" # Utilizes GPU when available.
print("Device: ",device)



Alfred = ANNModel(vocab_size, 0.01)



Alfred.training(train_x_tensor, train_y_tensor, validation_x_tensor, validation_y_tensor, 150, device)

currentTime = time.strftime("%Y%m%d_%H:%M:%S")
model_name = f"Alfred_{currentTime}.pth"


print(f"Saving as: {model_name}")

torch.save(Alfred.best_model.state_dict(), f"/content/drive/MyDrive/Data/{model_name}")
with open(f"/content/drive/MyDrive/Data/{model_name}_History.txt", 'w') as file:
    file.write(Alfred.getHistoryAsString())



Device:  cuda
Best model accuracy 0.475
Best model found
Epoch: 1
Training Loss: 0.6931809782981873
Validation loss: 0.6960415840148926
Best Loss: 0.6960415840148926
Validation accuracy: 0.475
Best model accuracy 0.82
Best model found
Epoch: 2
Training Loss: 0.6831055879592896
Validation loss: 0.6258426904678345
Best Loss: 0.6258426904678345
Validation accuracy: 0.82
Best model accuracy 0.715
Best model found
Epoch: 3
Training Loss: 0.5611822605133057
Validation loss: 0.5996713638305664
Best Loss: 0.5996713638305664
Validation accuracy: 0.715
Best model accuracy 0.84
Best model found
Epoch: 4
Training Loss: 0.3178817331790924
Validation loss: 0.4102810323238373
Best Loss: 0.4102810323238373
Validation accuracy: 0.84
Epoch: 5
Training Loss: 0.07954699546098709
Validation loss: 0.5239108204841614
Best Loss: 0.4102810323238373
Validation accuracy: 0.77
Epoch: 6
Training Loss: 0.009759440086781979
Validation loss: 1.0295990705490112
Best Loss: 0.4102810323238373
Validation accuracy: 0.71
E

In [ ]:
#BIG data training

BIG_TrainingData = pd.DataFrame({'index': pd.RangeIndex(len(BIG_training_data)),'Sentence': BIG_training_data, 'Class': BIG_training_labels})
BIG_ValidationData = pd.DataFrame({'index': pd.RangeIndex(len(BIG_validation_data)), 'Sentence': BIG_validation_data, 'Class': BIG_validation_labels})
BIG_TestData = pd.DataFrame({'index': pd.RangeIndex(len(BIG_test_data)), 'Sentence': BIG_test_data, 'Class': BIG_test_labels})


#Pre-Processing the data
BIG_processedTrainingData = preprocess_pandas(BIG_TrainingData, columns)
BIG_processedValidationData = preprocess_pandas(BIG_ValidationData, columns)
BIG_processedTestData = preprocess_pandas(BIG_TestData, columns)





BIG_anntraining_data = word_vectorizer.fit_transform(BIG_processedTrainingData['Sentence']) # transform texts to sparse matrix
BIG_anntraining_data = BIG_anntraining_data.todense() # convert to dense matrix for Pytorch
BIG_vocab_size = len(word_vectorizer.vocabulary_)
BIG_annvalidation_data = word_vectorizer.transform(BIG_processedValidationData['Sentence'])
BIG_annvalidation_data = BIG_annvalidation_data.todense()
BIG_anntest_data = word_vectorizer.transform(BIG_processedTestData['Sentence'])
BIG_anntest_data = BIG_anntest_data.todense()
BIG_train_x_tensor = torch.from_numpy(np.array(BIG_anntraining_data)).type(torch.FloatTensor)
BIG_train_y_tensor = torch.from_numpy(np.array(BIG_training_labels)).type(torch.LongTensor)
BIG_validation_x_tensor = torch.from_numpy(np.array(BIG_annvalidation_data)).type(torch.FloatTensor)
BIG_validation_y_tensor = torch.from_numpy(np.array(BIG_validation_labels)).type(torch.LongTensor)
BIG_test_x_tensor = torch.from_numpy(np.array(BIG_anntest_data)).type(torch.FloatTensor)
BIG_test_y_tensor = torch.from_numpy(np.array(BIG_test_labels)).type(torch.LongTensor)




device = "cuda" if torch.cuda.is_available() else "cpu" # Utilizes GPU when available.
print("Device: ",device)



BIG_Alfred = ANNModel(BIG_vocab_size, 0.01)



BIG_Alfred.training(BIG_train_x_tensor, BIG_train_y_tensor, BIG_validation_x_tensor, BIG_validation_y_tensor, 100, device)

currentTime = time.strftime("%Y%m%d_%H:%M:%S")
model_name = f"Big_Alfred_{currentTime}.pth"


print(f"Saving as: {model_name}")

torch.save(BIG_Alfred.best_model.state_dict(), f"/content/drive/MyDrive/Data/{model_name}")
with open(f"/content/drive/MyDrive/Data/{model_name}_History.txt", 'w') as file:
    file.write(BIG_Alfred.getHistoryAsString())


Device:  cuda
Best model accuracy 0.6148
Best model found
Epoch: 1
Training Loss: 0.6933538913726807
Validation loss: 0.69671630859375
Best Loss: 0.6521293520927429
Validation accuracy: 0.6148
Epoch: 2
Training Loss: 0.6817252039909363
Validation loss: 0.6158673167228699
Best Loss: 0.6521293520927429
Validation accuracy: 0.6148
Best model accuracy 0.6148
Best model found
Epoch: 3
Training Loss: 0.5366295576095581
Validation loss: 0.6090210676193237
Best Loss: 0.6098462343215942
Validation accuracy: 0.6148
Epoch: 4
Training Loss: 0.29301580786705017
Validation loss: 0.42191848158836365
Best Loss: 0.6098462343215942
Validation accuracy: 0.6148
Best model accuracy 0.6148
Best model found
Epoch: 5
Training Loss: 0.08137450367212296
Validation loss: 0.506181538105011
Best Loss: 0.574897050857544
Validation accuracy: 0.6148
Best model accuracy 0.6226
Best model found
Epoch: 6
Training Loss: 0.010512332431972027
Validation loss: 0.9933529496192932
Best Loss: 0.4997919797897339
Validation accu

In [ ]:

#Little data Testing


with torch.no_grad():
    Alfred.best_model.eval()
    Alfred.best_model.to(device)
    test_x_tensor = test_x_tensor.to(device)
    test_y_tensor = test_y_tensor.to(device)
    prediction = Alfred.best_model(test_x_tensor)
    correct = 0
    for i in range(len(test_y_tensor)):
        predicted = torch.argmax(prediction[i])
        if predicted == test_y_tensor[i]:
            correct += 1
    accuracy = correct / len(test_y_tensor)
    print(f"Accuracy: {accuracy}")





Accuracy: 0.81


In [ ]:
# BIG data testing



with torch.no_grad():
    BIG_Alfred.best_model.eval()
    BIG_Alfred.best_model.to(device)
    BIG_test_x_tensor = BIG_test_x_tensor.to(device)
    BIG_test_y_tensor = BIG_test_y_tensor.to(device)
    prediction = BIG_Alfred.best_model(BIG_test_x_tensor)
    correct = 0
    for i in range(len(BIG_test_y_tensor)):
        predicted = torch.argmax(prediction[i])
        if predicted == BIG_test_y_tensor[i]:
            correct += 1
    accuracy = correct / len(BIG_test_y_tensor)
    print(f"Accuracy: {accuracy}")


Accuracy: 0.874


In [ ]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses
pip install datasets
pip install evaluate


Training the model

In [ ]:
print(validation_data[0])

Piece of trash.


## Task 1.2: Transformers Implementation


In [ ]:
from transformers import AutoModelForSequenceClassification, DistilBertModel
from torch.optim import AdamW
from torch.nn.functional import cross_entropy
from transformers import AutoTokenizer, TrainingArguments
import numpy as np
import tensorflow as tf
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer
from datasets import load_dataset, Dataset, DatasetDict, table
import evaluate

     index                                           Sentence  Class
0        0  a must study for anyone interested in the "wor...      0
1        1  i purcashed this for the car charger and it do...      0
2        2  i'm happy about this purchase- good quality an...      1
3        3                             soyo technology sucks.      0
4        4                    very dissapointing performance.      0
..     ...                                                ...    ...
595    595  the calls drop, the phone comes on and off at ...      0
596    596  lately they have been extremely nice and helpf...      1
597    597  i recommend igo to anyone with different brand...      1
598    598               none of it works, just don't buy it.      0
599    599                         chinese forgeries abound!.      0

[600 rows x 3 columns]
     index                                           Sentence  Class
0        0  a must study for anyone interested in the "wor...      0
1        1

In [ ]:



trainingDatasetForTransformer = Dataset.from_dict({'text': training_data.tolist(), 'label': training_labels.tolist()})
validationDatasetForTransformer = Dataset.from_dict({'text': validation_data.tolist(), 'label': validation_labels.tolist()})
testDatasetForTransformer = Dataset.from_dict({'text': test_data.tolist(), 'label': test_labels.tolist()})




In [ ]:
#Angelo Name

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

trainingDatasetForTransformer = trainingDatasetForTransformer.map(tokenize, batched=True)
validationDatasetForTransformer = validationDatasetForTransformer.map(tokenize, batched=True)
testDatasetForTransformer = testDatasetForTransformer.map(tokenize, batched=True)


#print(trainingDatasetForTransformer[0]["attention_mask"])
#print(trainingDatasetForTransformer[0]["input_ids"])





Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("roberta-base")


def tokenize1(examples):
    return tokenizer1(examples["text"], padding="max_length", truncation=True)

trainingDatasetForTransformer1 = trainingDatasetForTransformer.map(tokenize1, batched=True)
validationDatasetForTransformer1 = validationDatasetForTransformer.map(tokenize1, batched=True)
testDatasetForTransformer1 = testDatasetForTransformer.map(tokenize1, batched=True)


#print(trainingDatasetForTransformer[0]["attention_mask"])
#print(trainingDatasetForTransformer[0]["input_ids"])




Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:


#Transformer Class

correct = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Angelo = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="amazon_output",
    eval_strategy="epoch",
    push_to_hub=False,
    num_train_epochs=10,

)


trainer = Trainer(
    model=Angelo,
    args=training_args,
    train_dataset=trainingDatasetForTransformer,
    eval_dataset=validationDatasetForTransformer,

    compute_metrics=compute_metrics,
)
trainer.train()





Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: m00n (ertveh-4-lule-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.278005,0.915000
2,No log,0.380628,0.915000
3,No log,0.398820,0.915000
4,No log,0.387652,0.925000
5,No log,0.419409,0.920000
6,No log,0.432654,0.915000
7,0.088100,0.439755,0.920000
8,0.088100,0.470740,0.915000
9,0.088100,0.475543,0.910000
10,0.088100,0.468922,0.915000


TrainOutput(global_step=750, training_loss=0.060671894550323485, metrics={'train_runtime': 434.4742, 'train_samples_per_second': 13.81, 'train_steps_per_second': 1.726, 'total_flos': 794804391936000.0, 'train_loss': 0.060671894550323485, 'epoch': 10.0})

In [ ]:
#Transformer Class

correct = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Angelo1 = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)


metric1 = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric1.compute(predictions=predictions, references=labels)


training_args1 = TrainingArguments(
    output_dir="roberta-base_amazon_output",
    eval_strategy="epoch",
    push_to_hub=False,
    num_train_epochs=10,

)


trainer1 = Trainer(
    model=Angelo1,
    args=training_args1,
    train_dataset=trainingDatasetForTransformer1,
    eval_dataset=validationDatasetForTransformer1,

    compute_metrics=compute_metrics,
)
trainer1.train()




Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.590037,0.685000
2,No log,0.303279,0.895000
3,No log,0.402757,0.920000
4,No log,0.503240,0.895000
5,No log,0.391011,0.930000
6,No log,0.515971,0.910000
7,0.259200,0.474767,0.920000
8,0.259200,0.431430,0.925000
9,0.259200,0.477206,0.925000
10,0.259200,0.481778,0.925000


TrainOutput(global_step=750, training_loss=0.18196627680460611, metrics={'train_runtime': 686.1247, 'train_samples_per_second': 8.745, 'train_steps_per_second': 1.093, 'total_flos': 1578666332160000.0, 'train_loss': 0.18196627680460611, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.46892234683036804, 'eval_accuracy': 0.915, 'eval_runtime': 3.0453, 'eval_samples_per_second': 65.676, 'eval_steps_per_second': 8.209, 'epoch': 10.0}


In [ ]:
eval_results1 = trainer1.evaluate()
print(f"Evaluation results: {eval_results1}")

Evaluation results: {'eval_loss': 0.4817781150341034, 'eval_accuracy': 0.925, 'eval_runtime': 4.6886, 'eval_samples_per_second': 42.656, 'eval_steps_per_second': 5.332, 'epoch': 10.0}


In [ ]:
trainer.predict(testDatasetForTransformer)

PredictionOutput(predictions=array([[-4.0287223 ,  4.03092   ],
       [ 4.6550198 , -4.581804  ],
       [ 4.5990443 , -4.573219  ],
       [ 3.4449742 , -3.2391732 ],
       [-3.33469   ,  3.3744228 ],
       [ 1.1028445 , -0.87944615],
       [ 4.6235476 , -4.5462875 ],
       [-4.1157465 ,  4.082939  ],
       [ 3.943691  , -3.7902472 ],
       [ 4.597421  , -4.4443626 ],
       [ 4.6033616 , -4.552198  ],
       [ 4.649841  , -4.5404773 ],
       [ 3.274695  , -2.9840846 ],
       [-4.114347  ,  4.0338264 ],
       [-4.1227975 ,  4.0005255 ],
       [ 4.451365  , -4.34954   ],
       [-3.9105988 ,  3.9494786 ],
       [ 4.603037  , -4.5097485 ],
       [-4.004021  ,  4.066724  ],
       [ 3.5346863 , -3.2163653 ],
       [-3.9903028 ,  3.951116  ],
       [-3.7936826 ,  3.788242  ],
       [ 4.613532  , -4.5201387 ],
       [ 4.6399918 , -4.5458393 ],
       [ 4.6624813 , -4.599609  ],
       [ 4.5873294 , -4.509691  ],
       [ 4.6688166 , -4.5540824 ],
       [ 4.4931173 , -4.44

In [ ]:
trainer1.predict(testDatasetForTransformer)

PredictionOutput(predictions=array([[-2.2700083 ,  2.1964583 ],
       [ 2.0621796 , -1.8629085 ],
       [-0.6006635 ,  0.5894978 ],
       [-2.0061996 ,  1.9640536 ],
       [-2.4592004 ,  2.3217485 ],
       [ 2.0968215 , -1.938482  ],
       [ 2.0939882 , -1.9084678 ],
       [-2.743352  ,  2.5441964 ],
       [ 2.0588472 , -1.8759656 ],
       [ 2.083549  , -1.9239538 ],
       [ 2.0905197 , -1.9033805 ],
       [ 2.0942092 , -1.9119331 ],
       [ 2.0944104 , -1.9052031 ],
       [-2.2178137 ,  2.148123  ],
       [-2.747356  ,  2.5284462 ],
       [-2.5261998 ,  2.3907654 ],
       [ 2.0354502 , -1.8219371 ],
       [ 1.413814  , -1.2216955 ],
       [-2.7750707 ,  2.5304885 ],
       [ 2.0637174 , -1.8592227 ],
       [-2.6439435 ,  2.4887516 ],
       [-2.1298215 ,  2.0288048 ],
       [-1.5874218 ,  1.5195099 ],
       [ 2.085502  , -1.9258776 ],
       [ 2.094691  , -1.9164593 ],
       [ 2.0943851 , -1.9140168 ],
       [ 2.0956247 , -1.9138644 ],
       [ 2.0806344 , -1.88

## Task 1.3: Comparison

In [ ]:







#

Tranformers:
Slower training time,
Better accuracy,
Uses adam optimizer as default





Performance: \\
Transformer higher accuracy. \\
Low resources: ANN. \\
Accuracy important: Transfomers. \\
TFIDF embedding utilized. \\
More layer to get higher accuracy. \\
Adding momentum to achieve better accuracy. \\
More training epochs. \\
Utilize the gpu to faster training. \\



Insight:
Training time longer than expected, especially in transformers. \\


# LARGE dataset

In [ ]:
import numpy
#LARGE data loading in.

BATCH_SIZE = 1000


# get data, pre-process and split
LARGE_data = pd.read_csv("/content/drive/MyDrive/Data/amazon_reviews_multilingual_US_v1_00.tsv", delimiter='\t', on_bad_lines='skip')
#print(type(data))
LARGE_data.columns = ['Class','Sentence']
LARGE_data['index'] = LARGE_data.index # add new column index
# split the data into training, validation, and test splits
LARGE_training_data, LARGE_valtest_data, LARGE_training_labels, LARGE_valtest_labels = train_test_split(
    LARGE_data['Sentence'].values.astype('U'),
    LARGE_data['Class'].values.astype('int32'),
    test_size=0.40,
    random_state=0,
    shuffle=True
)

LARGE_test_data, LARGE_validation_data, LARGE_test_labels, LARGE_validation_labels = train_test_split(
    LARGE_valtest_data,
    LARGE_valtest_labels,
    test_size=0.50,
    random_state=0,
    shuffle=True
)



LARGE_TrainingData = pd.DataFrame({'index': pd.RangeIndex(len(LARGE_training_data)),'Sentence': LARGE_training_data, 'Class': LARGE_training_labels})
LARGE_ValidationData = pd.DataFrame({'index': pd.RangeIndex(len(LARGE_validation_data)), 'Sentence': LARGE_validation_data, 'Class': LARGE_validation_labels})
LARGE_TestData = pd.DataFrame({'index': pd.RangeIndex(len(LARGE_test_data)), 'Sentence': LARGE_test_data, 'Class': LARGE_test_labels})


#Pre-Processing the data
LARGE_processedTrainingData = preprocess_pandas(LARGE_TrainingData, columns)
LARGE_processedValidationData = preprocess_pandas(LARGE_ValidationData, columns)
LARGE_processedTestData = preprocess_pandas(LARGE_TestData, columns)


LARGE_anntraining_data = word_vectorizer.fit_transform(LARGE_processedTrainingData['Sentence']) # transform texts to sparse matrix
LARGE_anntraining_data = LARGE_anntraining_data.todense() # convert to dense matrix for Pytorch
LARGE_vocab_size = len(word_vectorizer.vocabulary_)
LARGE_annvalidation_data = word_vectorizer.transform(BIG_processedValidationData['Sentence'])
LARGE_annvalidation_data = LARGE_annvalidation_data.todense()
LARGE_anntest_data = word_vectorizer.transform(LARGE_processedTestData['Sentence'])
LARGE_anntest_data = LARGE_anntest_data.todense()
LARGE_train_x_tensor = torch.from_numpy(np.array(LARGE_anntraining_data)).type(torch.FloatTensor)
LARGE_train_y_tensor = torch.from_numpy(np.array(LARGE_training_labels)).type(torch.LongTensor)
LARGE_validation_x_tensor = torch.from_numpy(np.array(LARGE_annvalidation_data)).type(torch.FloatTensor)
LARGE_validation_y_tensor = torch.from_numpy(np.array(LARGE_validation_labels)).type(torch.LongTensor)
LARGE_test_x_tensor = torch.from_numpy(np.array(LARGE_anntest_data)).type(torch.FloatTensor)
LARGE_test_y_tensor = torch.from_numpy(np.array(LARGE_test_labels)).type(torch.LongTensor)



NameError: name 'pd' is not defined

In [ ]:
class LARGE_ANNModel():

    training_losses, validation_losses = [], []
    best_model = None

    def __init__(self, in_features, learning_rate):
        self.model = nn.Sequential(
            nn.Linear(in_features, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 5)
        )
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay = 0.0009)

        self.loss_fn = torch.nn.CrossEntropyLoss()



    def training(self, train_data, train_labels, validation_data, validation_labels, epochs, device):
        num_epochs = epochs
        best_loss = float("inf")

        self.model.train()
        self.model.to(device)
        train_data = train_data.to(device)
        train_labels = train_labels.to(device)
        validation_data = validation_data.to(device)
        validation_labels = validation_labels.to(device)



        for epoch in range(num_epochs):
            batch_loss = []
            for batch in enumerate(len(train_data) // BATCH_SIZE):
                training_loss = 0

                # To be adjusted based off how dataloaders are implemented. For now assume each batch holds the images and the labels
                # for i, data in enumerate(train_data):

                prediction = self.model(train_data[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)])

                self.optimizer.zero_grad()

                loss = self.loss_fn(prediction, train_labels[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)])

                loss.backward()

                self.optimizer.step()
                batch_loss.append(loss.item())
            self.training_losses.append(numpy.mean(batch_loss))

            correct = 0
            batch_loss = []
            for batch in enumerate(len(validation_data) // BATCH_SIZE):
                validation_loss = 0

                # Validation loop
                with torch.no_grad():
                    prediction = self.model(validation_data[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)])

                    loss = self.loss_fn((prediction), (validation_labels[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)]))
                    batch_loss.append(loss.item())
                    for i in range(BATCH_SIZE):
                        predicted = torch.argmax(prediction[i])
                        if predicted == validation_labels[batch*BATCH_SIZE+i]:
                            correct += 1

            self.validation_losses.append(numpy.mean(batch_loss))
            accuracy = correct / len(validation_labels)

            if validation_loss[epoch] < best_loss:
                    best_loss = validation_loss
                    self.best_model = copy.deepcopy(self.model)
                    print("Best model accuracy", accuracy)
                    print("Best model found")

            print(f"Epoch: {epoch+1}\nTraining Loss: {self.training_losses[epoch]}\nValidation loss: {self.validation_losses[epoch]}\nBest Loss: {best_loss}\nValidation accuracy: {accuracy}")

    def getHistoryAsString(self):
        text = ""

        for i in range(len(self.training_losses)):
            text += f"Epoch: {i+1}\nTraining Loss: {self.training_losses[i]}\nValidation loss: {self.validation_losses[i]}\n\n"
        return text

Training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Utilizes GPU when available.
print("Device: ",device)



LARGE_model = LARGE_ANNModel(LARGE_vocab_size, 0.01)



LARGE_model.training(LARGE_train_x_tensor, LARGE_train_y_tensor, LARGE_validation_x_tensor, LARGE_validation_y_tensor, 100, device)

currentTime = time.strftime("%Y%m%d_%H:%M:%S")
model_name = f"LARGE_model_{currentTime}.pth"


print(f"Saving as: {model_name}")

torch.save(LARGE_model.best_model.state_dict(), f"/content/drive/MyDrive/Data/{model_name}")
with open(f"/content/drive/MyDrive/Data/{model_name}_History.txt", 'w') as file:
    file.write(LARGE_model.getHistoryAsString())